In [1]:
import pandas as pd
import meterdatalogic as ml

TZ = "Australia/Brisbane"


In [2]:
# 7 days of half-hourly data for one NMI
rng = pd.date_range("2025-01-01", periods=48*7, freq="30min", tz=TZ)
df = pd.DataFrame({
    "t_start": rng,
    "nmi": "Q1234567890",
    "channel": "E1",
    "kwh": 0.5
})
df.head()


,t_start,nmi,channel,kwh
0,2025-01-01 00:00:00+10:00,Q1234567890,E1,0.5
1,2025-01-01 00:30:00+10:00,Q1234567890,E1,0.5
2,2025-01-01 01:00:00+10:00,Q1234567890,E1,0.5
3,2025-01-01 01:30:00+10:00,Q1234567890,E1,0.5
4,2025-01-01 02:00:00+10:00,Q1234567890,E1,0.5


In [3]:
canon_df = ml.ingest.from_nem12("../examples/data/Sample1.csv", tz="Australia/Brisbane")
ml.validate.assert_canon(canon_df)

print(canon_df.index.tz, df.index.name)
print(canon_df.head())
print(canon_df["flow"].value_counts())
canon_df

Australia/Brisbane None
                               nmi channel         flow    kwh  cadence_min
t_start                                                                    
2014-05-06 00:00:00+10:00  SAMPLE1      E1  grid_import  0.144           30
2014-05-06 00:30:00+10:00  SAMPLE1      E1  grid_import  0.163           30
2014-05-06 01:00:00+10:00  SAMPLE1      E1  grid_import  0.106           30
2014-05-06 01:30:00+10:00  SAMPLE1      E1  grid_import  0.144           30
2014-05-06 02:00:00+10:00  SAMPLE1      E1  grid_import  0.169           30
flow
grid_import    35136
Name: count, dtype: int64


/Users/tyler/Developer/mdt-django-poc/meterdatalogic/meterdatalogic/ingest.py:30: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_infer_group_cadence)


,nmi,channel,flow,kwh,cadence_min
t_start,,,,,
2014-05-06 00:00:00+10:00,SAMPLE1,E1,grid_import,0.144,30
2014-05-06 00:30:00+10:00,SAMPLE1,E1,grid_import,0.163,30
2014-05-06 01:00:00+10:00,SAMPLE1,E1,grid_import,0.106,30
2014-05-06 01:30:00+10:00,SAMPLE1,E1,grid_import,0.144,30
2014-05-06 02:00:00+10:00,SAMPLE1,E1,grid_import,0.169,30
...,...,...,...,...,...
2016-05-06 21:30:00+10:00,SAMPLE1,E1,grid_import,0.394,30
2016-05-06 22:00:00+10:00,SAMPLE1,E1,grid_import,0.356,30
2016-05-06 22:30:00+10:00,SAMPLE1,E1,grid_import,0.369,30


In [4]:
# Daily energy by flow (wide columns)
import meterdatalogic as ml

daily = ml.transform.aggregate(canon_df, freq="1D", groupby="flow", pivot=True)
daily.head()

flow,grid_import
t_start,
2014-05-06 00:00:00+10:00,25.440
2014-05-07 00:00:00+10:00,25.228
2014-05-08 00:00:00+10:00,28.367
2014-05-09 00:00:00+10:00,23.671
2014-05-10 00:00:00+10:00,29.284


In [5]:
# Monthly peak demand (MF 16:00–21:00) in kW
import meterdatalogic as ml

demand = ml.transform.aggregate(
    canon_df,
    freq="1MS",
    flows=["grid_import"],
    metric="kW",
    stat="max",
    out_col="demand_kw",
    window_start="16:00",
    window_end="21:00",
    window_days="MF",
)
demand.tail()

,demand_kw
t_start,
2016-01-01 00:00:00+10:00,3.738
2016-02-01 00:00:00+10:00,4.838
2016-03-01 00:00:00+10:00,4.550
2016-04-01 00:00:00+10:00,7.724
2016-05-01 00:00:00+10:00,3.476


In [6]:
# Time-of-Use bins (month + one column per band name)
import meterdatalogic as ml

bands = [
    {"name": "off", "start": "00:00", "end": "16:00"},
    {"name": "peak", "start": "16:00", "end": "21:00"},
    {"name": "shoulder", "start": "21:00", "end": "24:00"},
]

tou = ml.transform.tou_bins(canon_df, bands)
tou.head()

band,month,off,peak,shoulder
0,2014-05,371.368,234.112,125.812
1,2014-06,449.213,266.161,156.129
2,2014-07,448.150,275.345,184.004
3,2014-08,413.570,190.108,105.303
4,2014-09,373.269,173.923,89.425


In [7]:
# Hourly tidy aggregation grouped by NMI, channel, and flow
import meterdatalogic as ml

hourly = ml.transform.aggregate(
    canon_df, freq="1H", groupby=["nmi", "channel", "flow"], value_col="kwh", pivot=False
)
hourly.head()

/Users/tyler/Developer/mdt-django-poc/meterdatalogic/meterdatalogic/transform.py:160: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample(freq, label=label, closed=closed)["_val"]


,nmi,channel,flow,t_start,kwh
0,SAMPLE1,E1,grid_import,2014-05-06 00:00:00+10:00,0.307
1,SAMPLE1,E1,grid_import,2014-05-06 01:00:00+10:00,0.250
2,SAMPLE1,E1,grid_import,2014-05-06 02:00:00+10:00,0.344
3,SAMPLE1,E1,grid_import,2014-05-06 03:00:00+10:00,0.294
4,SAMPLE1,E1,grid_import,2014-05-06 04:00:00+10:00,0.338


In [8]:
# Average day profile by intra-day slot (HH:MM), per flow
import pandas as pd

_prof = canon_df.copy()
_prof["slot"] = pd.DatetimeIndex(_prof.index).strftime("%H:%M")
prof = (
    _prof.groupby(["slot", "flow"])['kwh']
    .mean()
    .unstack("flow")
    .fillna(0.0)
    .reset_index()
)
prof.head()

flow,slot,grid_import
0,00:00,0.228359
1,00:30,0.208526
2,01:00,0.186454
3,01:30,0.180960
4,02:00,0.189959
